In [1]:
import os
import sys
import csv
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, UpSampling2D, Conv2D, Conv2DTranspose, Reshape, Input
from tensorflow.keras import optimizers
from efficientnet.tfkeras import EfficientNetB2, preprocess_input
import sklearn
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import OneClassSVM
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score
import cv2
import MVTecAD_loaddata as mv
import pandas as pd
res = []
def OCSVM_split(X, y, train_data):
    tuned_parameters = [
    {'nu': [2**-3, 2**-4, 2**-5,2**-6,2**-7], 'kernel': ['rbf'], 'gamma': [10**-3, 10**-4, 10**-5, 10**-6, 10**-7]},
    ]
    score = 'accuracy'
    clf = GridSearchCV(
        OneClassSVM(),
        tuned_parameters,
        cv=5,
        scoring=score)
#         scoring='%s_weighted' % score )
    clf.fit(X, y)
    svm = OneClassSVM(nu=clf.best_params_['nu'], gamma=clf.best_params_['gamma'])
    svm.fit(X,y)
    sv_num = svm.support_
    sv = train_data[sv_num,:]
    return clf, svm, sv

def OCSVM2(clf, svm, X, x_test, y, y_test):
    svm.fit(X, y)
    a = svm.decision_function(x_test)
    b = svm.predict(x_test)
    r1 = roc_auc_score(y_test,a)
    r2 = f1_score(y_test,b)
    r3 = accuracy_score(y_test,b)
    print('roc_auc : ',r1)
    print('f1_score : ', r2)
    print('accuracy_score : ', r3)
    print('best_params : ', clf.best_params_)
    result = [r1,r2,r3]
    return result
def OCSVM(X, x_test, y, y_test):
    tuned_parameters = [
        {'nu': [2**-3, 2**-4, 2**-5, 2**-6, 2**-7], 'kernel': ['rbf'], 'gamma': [10**-3, 10**-4, 10**-5, 10**-6, 10**-7]},
    ]
    score = 'f1'
    clf = GridSearchCV(
        OneClassSVM(),
        tuned_parameters,
        cv=5,
        scoring='%s_weighted' % score)
          
    clf.fit(X, y)
          
    a = clf.decision_function(x_test)
    b = clf.predict(x_test)
    print('roc_auc : ', roc_auc_score(y_test, a))
    print('f1_score : ', f1_score(y_test, b))
    print('accuracy_score : ', accuracy_score(y_test, b))
    print('best_params : ', clf.best_params_)

def model_fit(X_train, X_test, SVs, model):
    model.fit(SVs, SVs, epochs=1, batch_size=16, shuffle=True)
    encoder = Model(model.layers[0].input, model.layers[-8].output)
    f_tr = encoder.predict(X_train, verbose=0)
    f_ts = encoder.predict(X_test, verbose=0)
    x_tr = np.reshape(f_tr, (f_tr.shape[0], f_tr.shape[1] * f_tr.shape[2] * f_tr.shape[3]))
    x_ts = np.reshape(f_ts, (f_ts.shape[0], f_ts.shape[1] * f_ts.shape[2] * f_ts.shape[3]))
    return x_tr, x_ts

def model_predict(X_train):
    input_shape = (256, 256, 3)
    inputs = Input(input_shape)

    base_model = EfficientNetB2(include_top=False, weights='imagenet')(inputs)
    x = Reshape((8, 8, 1408))(base_model)
    x = Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same')(x)
    x = Conv2DTranspose(64, kernel_size=(3, 3), activation='relu', padding='same', strides=2)(x)
    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2DTranspose(64, kernel_size=(3, 3), activation='relu', padding='same', strides=2)(x)
    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2DTranspose(64, kernel_size=(3, 3), activation='relu', padding='same', strides=2)(x)
    outputs = Conv2D(3, kernel_size=(3, 3), activation='sigmoid', padding='same')(x)
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=optimizers.Adam(learning_rate=1e-4), loss="mean_squared_error")
    
    encoder = Model(model.layers[0].input, model.layers[-8].output)
    f_tr = encoder.predict(X_train, verbose=0)
    x_tr = np.reshape(f_tr, (f_tr.shape[0], f_tr.shape[1] * f_tr.shape[2] * f_tr.shape[3]))
    return model, x_tr

In [2]:
input_shape = (256, 256, 3)
inputs = Input(input_shape)

base_model = EfficientNetB2(include_top=False, weights='imagenet')(inputs)
x = Reshape((8, 8, 1408))(base_model)
x = Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = Conv2DTranspose(64, kernel_size=(3, 3), activation='relu', padding='same', strides=2)(x)
x = UpSampling2D(size=(2, 2))(x)
x = Conv2DTranspose(64, kernel_size=(3, 3), activation='relu', padding='same', strides=2)(x)
x = UpSampling2D(size=(2, 2))(x)
x = Conv2DTranspose(64, kernel_size=(3, 3), activation='relu', padding='same', strides=2)(x)
outputs = Conv2D(3, kernel_size=(3, 3), activation='sigmoid', padding='same')(x)
model = Model(inputs=inputs, outputs=outputs)
model.compile(optimizer=optimizers.Adam(learning_rate=1e-4), loss="mean_squared_error")
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 efficientnet-b2 (Functional  (None, None, None, 1408)  7768562  
 )                                                               
                                                                 
 reshape (Reshape)           (None, 8, 8, 1408)        0         
                                                                 
 conv2d (Conv2D)             (None, 8, 8, 256)         3244288   
                                                                 
 conv2d_transpose (Conv2DTra  (None, 16, 16, 64)       147520    
 nspose)                                                         
                                                                 
 up_sampling2d (UpSampling2D  (None, 32, 32, 64)       0     

In [2]:
X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.bottle()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.cable()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.capsule()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.carpet()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.grid()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.hazelnut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.leather()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.metal_nut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.pill()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.screw()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.tile()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.toothbrush()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.transistor()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.wood()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.zipper()

model, f_train = model_predict(X_train)
clf, svm, sv = OCSVM_split(f_train, y_tr, X_train)
f_train, f_test = model_fit(X_train, X_test, sv, model)
result = OCSVM2(clf, svm, f_train, f_test, y_tr, y_ts)
res.append(result)

1/1 [==============================] - 20s 20s/step - loss: 0.1329
roc_auc :  0.9888888888888889
f1_score :  0.9230769230769231
accuracy_score :  0.963855421686747
best_params :  {'gamma': 1e-06, 'kernel': 'rbf', 'nu': 0.015625}


In [3]:
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.bottle()
X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.cable()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.capsule()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.carpet()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.grid()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.hazelnut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.leather()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.metal_nut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.pill()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.screw()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.tile()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.toothbrush()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.transistor()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.wood()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.zipper()

model, f_train = model_predict(X_train)
clf, svm, sv = OCSVM_split(f_train, y_tr, X_train)
f_train, f_test = model_fit(X_train, X_test, sv, model)
result = OCSVM2(clf, svm, f_train, f_test, y_tr, y_ts)
res.append(result)

1/1 [==============================] - 31s 31s/step - loss: 0.0531
roc_auc :  0.8969722455845248
f1_score :  0.759124087591241
accuracy_score :  0.7642857142857142
best_params :  {'gamma': 1e-06, 'kernel': 'rbf', 'nu': 0.015625}


In [4]:
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.bottle()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.cable()
X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.capsule()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.carpet()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.grid()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.hazelnut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.leather()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.metal_nut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.pill()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.screw()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.tile()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.toothbrush()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.transistor()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.wood()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.zipper()

model, f_train = model_predict(X_train)
clf, svm, sv = OCSVM_split(f_train, y_tr, X_train)
f_train, f_test = model_fit(X_train, X_test, sv, model)
result = OCSVM2(clf, svm, f_train, f_test, y_tr, y_ts)
res.append(result)

1/1 [==============================] - 22s 22s/step - loss: 0.0976
roc_auc :  0.7179896290386917
f1_score :  0.37606837606837606
accuracy_score :  0.44696969696969696
best_params :  {'gamma': 1e-06, 'kernel': 'rbf', 'nu': 0.0078125}


In [5]:
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.bottle()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.cable()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.capsule()
X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.carpet()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.grid()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.hazelnut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.leather()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.metal_nut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.pill()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.screw()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.tile()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.toothbrush()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.transistor()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.wood()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.zipper()

model, f_train = model_predict(X_train)
clf, svm, sv = OCSVM_split(f_train, y_tr, X_train)
f_train, f_test = model_fit(X_train, X_test, sv, model)
result = OCSVM2(clf, svm, f_train, f_test, y_tr, y_ts)
res.append(result)

1/1 [==============================] - 23s 23s/step - loss: 0.0390
roc_auc :  0.8756613756613756
f1_score :  0.5423728813559322
accuracy_score :  0.8014705882352942
best_params :  {'gamma': 1e-05, 'kernel': 'rbf', 'nu': 0.015625}


In [6]:
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.bottle()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.cable()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.capsule()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.carpet()
X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.grid()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.hazelnut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.leather()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.metal_nut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.pill()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.screw()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.tile()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.toothbrush()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.transistor()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.wood()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.zipper()

model, f_train = model_predict(X_train)
clf, svm, sv = OCSVM_split(f_train, y_tr, X_train)
f_train, f_test = model_fit(X_train, X_test, sv, model)
result = OCSVM2(clf, svm, f_train, f_test, y_tr, y_ts)
res.append(result)

1/1 [==============================] - 22s 22s/step - loss: 0.0309
roc_auc :  0.4845446950710109
f1_score :  0.39534883720930236
accuracy_score :  0.3333333333333333
best_params :  {'gamma': 1e-06, 'kernel': 'rbf', 'nu': 0.015625}


In [7]:
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.bottle()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.cable()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.capsule()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.carpet()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.grid()
X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.hazelnut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.leather()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.metal_nut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.pill()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.screw()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.tile()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.toothbrush()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.transistor()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.wood()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.zipper()

model, f_train = model_predict(X_train)
clf, svm, sv = OCSVM_split(f_train, y_tr, X_train)
f_train, f_test = model_fit(X_train, X_test, sv, model)
result = OCSVM2(clf, svm, f_train, f_test, y_tr, y_ts)
res.append(result)

1/1 [==============================] - 25s 25s/step - loss: 0.1065
roc_auc :  0.8042857142857143
f1_score :  0.6324786324786325
accuracy_score :  0.6090909090909091
best_params :  {'gamma': 1e-07, 'kernel': 'rbf', 'nu': 0.015625}


In [8]:
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.bottle()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.cable()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.capsule()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.carpet()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.grid()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.hazelnut()
X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.leather()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.metal_nut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.pill()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.screw()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.tile()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.toothbrush()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.transistor()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.wood()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.zipper()

model, f_train = model_predict(X_train)
clf, svm, sv = OCSVM_split(f_train, y_tr, X_train)
f_train, f_test = model_fit(X_train, X_test, sv, model)
result = OCSVM2(clf, svm, f_train, f_test, y_tr, y_ts)
res.append(result)

1/1 [==============================] - 23s 23s/step - loss: 0.0498
roc_auc :  0.9894701086956522
f1_score :  0.8275862068965517
accuracy_score :  0.9193548387096774
best_params :  {'gamma': 1e-05, 'kernel': 'rbf', 'nu': 0.015625}


In [9]:
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.bottle()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.cable()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.capsule()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.carpet()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.grid()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.hazelnut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.leather()
X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.metal_nut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.pill()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.screw()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.tile()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.toothbrush()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.transistor()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.wood()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.zipper()

model, f_train = model_predict(X_train)
clf, svm, sv = OCSVM_split(f_train, y_tr, X_train)
f_train, f_test = model_fit(X_train, X_test, sv, model)
result = OCSVM2(clf, svm, f_train, f_test, y_tr, y_ts)
res.append(result)

1/1 [==============================] - 23s 23s/step - loss: 0.0995
roc_auc :  0.9345063538611926
f1_score :  0.6027397260273972
accuracy_score :  0.7478260869565218
best_params :  {'gamma': 1e-06, 'kernel': 'rbf', 'nu': 0.0078125}


In [10]:
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.bottle()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.cable()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.capsule()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.carpet()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.grid()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.hazelnut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.leather()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.metal_nut()
X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.pill()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.screw()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.tile()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.toothbrush()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.transistor()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.wood()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.zipper()

model, f_train = model_predict(X_train)
clf, svm, sv = OCSVM_split(f_train, y_tr, X_train)
f_train, f_test = model_fit(X_train, X_test, sv, model)
result = OCSVM2(clf, svm, f_train, f_test, y_tr, y_ts)
res.append(result)

1/1 [==============================] - 22s 22s/step - loss: 0.1218
roc_auc :  0.7067648663393344
f1_score :  0.2911392405063291
accuracy_score :  0.32934131736526945
best_params :  {'gamma': 1e-06, 'kernel': 'rbf', 'nu': 0.0078125}


In [11]:
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.bottle()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.cable()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.capsule()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.carpet()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.grid()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.hazelnut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.leather()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.metal_nut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.pill()
X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.screw()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.tile()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.toothbrush()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.transistor()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.wood()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.zipper()

model, f_train = model_predict(X_train)
clf, svm, sv = OCSVM_split(f_train, y_tr, X_train)
f_train, f_test = model_fit(X_train, X_test, sv, model)
result = OCSVM2(clf, svm, f_train, f_test, y_tr, y_ts)
res.append(result)

2/2 [==============================] - 32s 2s/step - loss: 0.0656
roc_auc :  0.3756917401106784
f1_score :  0.37569060773480667
accuracy_score :  0.29375
best_params :  {'gamma': 1e-06, 'kernel': 'rbf', 'nu': 0.03125}


In [12]:
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.bottle()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.cable()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.capsule()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.carpet()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.grid()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.hazelnut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.leather()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.metal_nut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.pill()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.screw()
X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.tile()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.toothbrush()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.transistor()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.wood()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.zipper()

model, f_train = model_predict(X_train)
clf, svm, sv = OCSVM_split(f_train, y_tr, X_train)
f_train, f_test = model_fit(X_train, X_test, sv, model)
result = OCSVM2(clf, svm, f_train, f_test, y_tr, y_ts)
res.append(result)

1/1 [==============================] - 24s 24s/step - loss: 0.0183
roc_auc :  0.939033189033189
f1_score :  0.8148148148148148
accuracy_score :  0.8717948717948718
best_params :  {'gamma': 1e-05, 'kernel': 'rbf', 'nu': 0.015625}


In [13]:
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.bottle()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.cable()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.capsule()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.carpet()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.grid()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.hazelnut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.leather()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.metal_nut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.pill()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.screw()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.tile()
X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.toothbrush()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.transistor()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.wood()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.zipper()

model, f_train = model_predict(X_train)
clf, svm, sv = OCSVM_split(f_train, y_tr, X_train)
f_train, f_test = model_fit(X_train, X_test, sv, model)
result = OCSVM2(clf, svm, f_train, f_test, y_tr, y_ts)
res.append(result)

1/1 [==============================] - 22s 22s/step - loss: 0.1420
roc_auc :  0.8194444444444444
f1_score :  0.6451612903225806
accuracy_score :  0.7380952380952381
best_params :  {'gamma': 1e-06, 'kernel': 'rbf', 'nu': 0.015625}


In [14]:
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.bottle()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.cable()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.capsule()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.carpet()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.grid()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.hazelnut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.leather()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.metal_nut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.pill()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.screw()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.tile()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.toothbrush()
X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.transistor()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.wood()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.zipper()

model, f_train = model_predict(X_train)
clf, svm, sv = OCSVM_split(f_train, y_tr, X_train)
f_train, f_test = model_fit(X_train, X_test, sv, model)
result = OCSVM2(clf, svm, f_train, f_test, y_tr, y_ts)
res.append(result)

1/1 [==============================] - 21s 21s/step - loss: 0.0703
roc_auc :  0.83875
f1_score :  0.8489208633093525
accuracy_score :  0.79
best_params :  {'gamma': 1e-07, 'kernel': 'rbf', 'nu': 0.015625}


In [15]:
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.bottle()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.cable()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.capsule()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.carpet()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.grid()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.hazelnut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.leather()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.metal_nut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.pill()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.screw()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.tile()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.toothbrush()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.transistor()
X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.wood()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.zipper()

model, f_train = model_predict(X_train)
clf, svm, sv = OCSVM_split(f_train, y_tr, X_train)
f_train, f_test = model_fit(X_train, X_test, sv, model)
result = OCSVM2(clf, svm, f_train, f_test, y_tr, y_ts)
res.append(result)

1/1 [==============================] - 21s 21s/step - loss: 0.0216
roc_auc :  0.9780701754385964
f1_score :  0.8571428571428571
accuracy_score :  0.9240506329113924
best_params :  {'gamma': 1e-06, 'kernel': 'rbf', 'nu': 0.0078125}


In [16]:
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.bottle()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.cable()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.capsule()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.carpet()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.grid()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.hazelnut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.leather()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.metal_nut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.pill()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.screw()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.tile()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.toothbrush()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.transistor()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.wood()
X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.zipper()

model, f_train = model_predict(X_train)
clf, svm, sv = OCSVM_split(f_train, y_tr, X_train)
f_train, f_test = model_fit(X_train, X_test, sv, model)
result = OCSVM2(clf, svm, f_train, f_test, y_tr, y_ts)
res.append(result)

1/1 [==============================] - 22s 22s/step - loss: 0.1124
roc_auc :  0.8300945378151261
f1_score :  0.49180327868852464
accuracy_score :  0.5894039735099338
best_params :  {'gamma': 1e-06, 'kernel': 'rbf', 'nu': 0.015625}


In [17]:
df = pd.DataFrame(res)
df.to_csv('DCAEOCSVM_SV.csv')